In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim


from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [2]:
gbin0 = np.load('/home/jua/corsika/run/energy_var/npz/0427/bin/gamma_energy_bin0_all.npz')
pbin0 = np.load('/home/jua/corsika/run/energy_var/npz/0427/bin/proton_energy_bin0_all.npz')

In [3]:
gim = gbin0['image'][:]
pim = pbin0['image'][:]
#gim = torch.tensor(gim).float() 
#pim = torch.tensor(gim).float() 
gim_label = torch.zeros([len(gim)])
pim_label = torch.ones([len(pim)])

In [4]:
train_images = np.vstack((gim[:int(len(gim)*0.7)],pim[:int(len(pim)*0.7)]))
test_images  = np.vstack((gim[int(len(gim)*0.7):],pim[int(len(pim)*0.7):]))
train_labels = np.hstack((gim_label[:int(len(gim)*0.7)],pim_label[:int(len(pim)*0.7)]))
test_labels  = np.hstack((gim_label[int(len(gim)*0.7):],pim_label[int(len(pim)*0.7):]))

In [5]:
train_images = torch.tensor(train_images).float()
train_labels = torch.tensor(train_labels).float()
test_images  = torch.tensor(test_images).float()
test_labels  = torch.tensor(test_labels).float()

In [6]:
train_sets = ([(train_images[i],train_labels[i])for i in range(len(train_labels))])
test_sets = ([(test_images[i],test_labels[i])for i in range(len(test_labels))])

In [31]:
trainloader = torch.utils.data.DataLoader(train_sets, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_sets, batch_size=4, shuffle=True, num_workers=2)

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(4, 64, kernel_size=7)
        self.batch1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5)    
        #self.batch2 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_drop = nn.Dropout2d()
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.batch1(x)
        x = self.pool(F.relu(self.conv2(x)))
        #x = self.batch2(x)
        x = x.view(-1, 64*6*6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [33]:
net = Net()

In [71]:
conv1 = nn.Conv2d(4, 64, kernel_size=7)
batch = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
conv2 = nn.Conv2d(64, 64, kernel_size=5)
conv2_drop = nn.Dropout2d()
pool = nn.MaxPool2d(2,2)
fc1 = nn.Linear(64*6*6, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 2)

In [62]:
x = pool(F.relu(conv2(x)))

In [65]:
x = batch(x)

In [67]:
x = x.view(-1,64*6*6)

In [74]:
x = F.relu(fc3(x))

In [31]:
net = Net()

In [75]:
x

tensor([[0.0000, 0.1087],
        [0.0000, 0.3208],
        [0.0000, 0.1081],
        [0.0766, 0.0990],
        [0.0000, 0.1112],
        [0.0000, 0.1100],
        [0.0123, 0.0874],
        [0.0000, 0.1094],
        [0.0000, 0.1094],
        [0.0000, 0.1090],
        [0.0000, 0.1106],
        [0.0368, 0.1559],
        [0.0000, 0.1090],
        [0.0000, 0.1083],
        [0.0000, 0.1177],
        [0.0000, 0.1116]], grad_fn=<ReluBackward0>)

In [32]:
#net.cuda()

In [14]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [34]:
model = net
since = time.time()
best_model_wts = model.state_dict()
best_acc = 0.0
train_losses = []
train_accuracy = []

In [35]:
scheduler = exp_lr_scheduler
scheduler.step()
model.train(True)
running_loss = 0.0
running_corrects= 0

In [36]:
for data in trainloader:
    inputs, labels = data
    inputs = Variable(inputs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = model(inputs)
    _, preds = torch.max(outputs.data, 1)
    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()
    running_loss += loss.data
    running_corrects += torch.sum(preds==labels.long())
    break

In [ ]:
        epoch_loss = running_loss/len(train_images)
        epoch_acc = float(running_corrects)/len(train_images)
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_acc)
        print('Train Loss: {:.4f}, Acc : {:.4f}'.format( epoch_loss, epoch_acc))
        if epoch_acc>best_acc:
            best_acc = epoch_acc
            best_model_sts = model.state_dict()
    print()
    time_elapsed = time.time() - since
    print("Training complete in {:.0f}s".format(time_elapsed//60, time_elapsed%60))
    print("Best Acc: {:4f}".format(best_acc))
    model.load_state_dict(best_model_wts)
    return model,train_losses, train_accuracy

In [34]:
def train_model(model, trainloader, criterion, optimizer, scheduler, num_epochs=1):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    train_losses = []
    train_accuracy = []
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs-1))
        print('-'*10)
        #optimizer.step()
        scheduler.step()
        model.train(True)
        running_loss = 0.0
        running_corrects= 0

        for data in trainloader:
            inputs, labels = data
            #inputs = Variable(inputs.cuda())
            #labels = Variable(labels.cuda())
            inputs = Variable(inputs)
            labels = Variable(labels)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            running_loss += loss.data
            running_corrects += torch.sum(preds==labels.long())
    
        epoch_loss = running_loss/len(train_images)
        epoch_acc = float(running_corrects)/len(train_images)
        train_losses.append(epoch_loss)
        train_accuracy.append(epoch_acc)
        print('Train Loss: {:.4f}, Acc : {:.4f}'.format( epoch_loss, epoch_acc))
        if epoch_acc>best_acc:
            best_acc = epoch_acc
            best_model_sts = model.state_dict()
    print()
    time_elapsed = time.time() - since
    print("Training complete in {:.0f}s".format(time_elapsed//60, time_elapsed%60))
    print("Best Acc: {:4f}".format(best_acc))
    model.load_state_dict(best_model_wts)
    return model,train_losses, train_accuracy

In [35]:
net, train_losses, train_accuracy = train_model(net, trainloader, criterion, optimizer, exp_lr_scheduler,5)

Epoch 0/4
----------
Train Loss: 0.1738, Acc : 0.5008
Epoch 1/4
----------
Train Loss: 0.1737, Acc : 0.5000
Epoch 2/4
----------
Train Loss: 0.1738, Acc : 0.4996
Epoch 3/4
----------
Train Loss: 0.1738, Acc : 0.4996
Epoch 4/4
----------
Train Loss: 0.1736, Acc : 0.5015

Training complete in 0s
Best Acc: 0.501536


In [18]:
running_loss = 0.0
running_corrects= 0
for data in trainloader:
    inputs, labels = data
    inputs = Variable(inputs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = net(inputs)
    _, preds = torch.max(outputs.data, 1)
    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()
    running_loss += loss.data
    running_corrects += torch.sum(preds==labels.long())
    break

In [38]:
net(inputs)

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddmmBackward>)

In [25]:
x = pool(F.relu(conv1(inputs)))

In [27]:
x = pool(F.relu(conv2(x)))

In [29]:
x = x.view(-1, 16*6*6)

In [31]:
x = F.relu(fc2(x))

In [34]:
x = fc3(x)

In [50]:
outputs = net(inputs)

In [53]:
loss =criterion(outputs, labels.long())

In [54]:
loss.data

tensor(1.7064)

In [24]:
conv1 = nn.Conv2d(4, 6, kernel_size=7)
conv2 = nn.Conv2d(6, 16, kernel_size=5)
conv2_drop = nn.Dropout2d()
pool = nn.MaxPool2d(2,2)
fc1 = nn.Linear(16*6*6, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 2)

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(4, 6, kernel_size=7)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*6*6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [43]:
net = Net()

In [44]:
net(inputs)

tensor([[-2.0792e+00,  6.7448e-01],
        [-4.9209e+00,  1.8319e-01],
        [-4.6647e-01, -5.5905e-01],
        [-6.3199e-01, -1.2556e-01],
        [-1.1133e+00, -2.2625e-02],
        [-5.1064e-01, -1.2066e-03],
        [-5.3242e-01, -2.2281e-01],
        [-7.4729e-01, -1.2443e-01],
        [-6.3355e-01, -2.9101e-01],
        [-3.7511e-01,  1.5296e-02],
        [-6.5265e+00,  2.5013e+00],
        [-9.4022e-01, -2.1924e-01],
        [-5.4421e-01,  5.0286e-01],
        [-4.1779e+00,  2.8329e+00],
        [-8.4374e+00,  4.1572e+00],
        [-9.9679e+00,  9.5558e+00]], grad_fn=<AddmmBackward>)